# Week1 HW1_Regression_EunseoKang_5조_강은서

### Q1) 이 경우에서는 Type 1 error가 높을까요? Type 2 Error가 높을까요? FP, FN과 관련지어 설명해주세요.

문제에서는 은행에서 설정한 threshold가 0.3으로 낮고, 따라서 acceptance rate가 매우 높다고 설명하고 있다. (즉 가입이 쉽다)
https://sumniya.tistory.com/26

FN(False Negative)는 실제 true 인 정답을 false라고 예측하는 것이므로 type 1 error,
FP(False Positive)은 실제 False인 정답을 true라고 예측하는 것이므로 type 2 error와 비슷한 개념이다.
이 문제에서는 threshold가 낮아 가입이 쉽기 때문에 (그만큼 가입이 어려운 사람들도 쉽게 가입시켜줄 가능성이 더 높기 때문에), FP가 더 높을 것이다. 즉 type 2 error가 더 높을 것이다.

### Q2-1) Accuracy, Precision and Recall이 무엇인지 TP,FP,FN,TN의 식으로 나타내 주세요.

Accuracy: 모델이 맞게 예측한 경우를 모두 고려하는 지표. 모델이 맞게 예측한 모든 경우의 비율
 Accuracy= (TP+TN)/(TP+TN+FP+FN)

Precision:모델이 True라고 분류한 것 중에서 실제 True인 것의 비율
 Precision= TP/(TP+FP)

Recall: 실제 True인 것 중에서 모델이 True라고 예측한 것의 비율
 Recall= TP/(TP+FN)
 
*Precision이나 Recall은 모두 실제 True인 정답을 모델이 True라고 예측한 경우에 관심이 있으나, 바라보고자 하는 관점만 다르다. Precision은 모델의 입장에서, 그리고 Recall은 실제 정답(data)의 입장에서 정답을 정답이라고 맞춘 경우를 생각하는 것!


### Q2-2) Precision and Recall에 관한 제가 든 예시가 아닌 실생활 예시 하나를 들어주세요.

흔한 예시일 수 있으나 **코로나 진단 키트** 예시
True를 코로나 확진판정, False를 음성 판정이라고 한다면, 
FN(실제로는 코로나 양성반응을 보이고 있는데 음성이라 예측)이 가지는 위험성이 FP(실제로는 음성반응을 보이고 있는데 양성이라 예측)하는 것보다 훨씬 크다. 따라서 FN이 더 치명적이므로, threshold를 낮춰 FN을 방지하는 것이 더 나을 것이다. 이 경우 Recall은 더 높아질 것이다. 

### Q3) 코드 따라해보고 주석 달기!
https://www.geeksforgeeks.org/identifying-handwritten-digits-using-logistic-regression-pytorch/

In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

In [2]:
# MNIST Dataset (Images and Labels)  #데이터 불러오기
train_dataset = dsets.MNIST(root ='./data', 
                            train = True, 
                            transform = transforms.ToTensor(), 
                            download = True) 

test_dataset = dsets.MNIST(root ='./data', 
                        train = False, #test set이니까 여기 False지정
                        transform = transforms.ToTensor()) 

In [3]:
# Hyper Parameters 
input_size = 784 #image size is 28*28
num_classes = 10 #y가 0~9까지 10개의 class로 나올 수 있으므로
num_epochs = 5 #전체 데이터 5번 train
batch_size = 100 
learning_rate = 0.001

In [4]:
# Dataset Loader (Input Pipline)  ##순서 그대로 카피해오면 batchsize지정이 안되어서 오류가 났음.
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)

In [5]:
class LogisticRegression(nn.Module): #우리가 사용할 모델 지정
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 

    def forward(self, x):   #define the forward pass
        out = self.linear(x) 
        return out 

In [6]:
model = LogisticRegression(input_size, num_classes) 

In [7]:
criterion = nn.CrossEntropyLoss() #loss함수 설정
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) #optimiser 설정

In [9]:
# Training the Model ###loss.data[0]이면 오류 발생(왜인지는 잘 모르겠어서 다른 분들 것을 참고했습니다)
for epoch in range(num_epochs): #epoch 5번만큼 반복
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28))  #numpy --> tensor 형태로 데이터 변환
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad()  #gradient 초기화해서 시작
        outputs = model(images)  #forward pass
        loss = criterion(outputs, labels) #Calculate the loss.
        loss.backward()  #backpropagation
        optimizer.step() #Update all weights.
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data)) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.0980
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.0203
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.9099
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.8568
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8032
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7502
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7163
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.5937
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.6220
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.4558
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.4221
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4155
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.4059
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4308
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.2552
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2506
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2884
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.3311
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2474
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1200
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [10]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28))  #numpy --> tensor 형태로 데이터 변환
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 

print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 

Accuracy of the model on the 10000 test images:  82 %


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


### Q4-1) 2-3 In[14] 코드에서 optim.SGD를 사용하지않고 코드를 짠다면 어떤 방식으로 짜야할까요? 설명해주세요.

SGD는 각각의 training sample들을 looping한 후에 가중치들이 업데이트 된다.
(확실하진 않지만ㅠㅠ)training sample들에 대한 for구문이 끝나기  weight를 업데이트 하는 코드를 넣어주면 될 것 같다.
weight= weight-(learning rate)* w.grad.data

### Q4-2) Gradient descent 코드를 구현하는 문제입니다. 2-3 In[14]의 코드에서 다른 optimizer(3개 ex)adam, rmsprop, sgd에 momentum 추가 등등)를 이용하여 결과값을 비교해주세요.

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [19]:
torch.manual_seed(1)

#### sgd (momentum default=0)

In [21]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1)
    
    def forward(self,x):
        return self.linear(x)
    
#모델 초기화
model = LinearRegressionModel()

#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs=1000
for epoch in range(nb_epochs + 1):
    
   
    prediction = model(x_train) #H(x)계산
    cost= F.mse_loss(prediction, y_train)#cost 계산
    optimizer.zero_grad() #cost로 H(x) 개선
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        params= list(model.parameters())
        W = params[0].item()
        b= params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))


Epoch    0/1000 W: -0.101, b: 0.508 Cost: 4.630286
Epoch  100/1000 W: 0.713, b: 0.653 Cost: 0.061555
Epoch  200/1000 W: 0.774, b: 0.514 Cost: 0.038037
Epoch  300/1000 W: 0.822, b: 0.404 Cost: 0.023505
Epoch  400/1000 W: 0.860, b: 0.317 Cost: 0.014525
Epoch  500/1000 W: 0.890, b: 0.250 Cost: 0.008975
Epoch  600/1000 W: 0.914, b: 0.196 Cost: 0.005546
Epoch  700/1000 W: 0.932, b: 0.154 Cost: 0.003427
Epoch  800/1000 W: 0.947, b: 0.121 Cost: 0.002118
Epoch  900/1000 W: 0.958, b: 0.095 Cost: 0.001309
Epoch 1000/1000 W: 0.967, b: 0.075 Cost: 0.000809


#### adam

In [22]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1)
    
    def forward(self,x):
        return self.linear(x)
    
#모델 초기화
model = LinearRegressionModel()

#optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.01)

nb_epochs=1000
for epoch in range(nb_epochs + 1):
    
    #H(x)계산
    prediction = model(x_train)
    
    #cost 계산
    cost= F.mse_loss(prediction, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        params= list(model.parameters())
        W = params[0].item()
        b= params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.931, b: 0.610 Cost: 13.291902
Epoch  100/1000 W: -0.119, b: 1.401 Cost: 1.568305
Epoch  200/1000 W: 0.204, b: 1.617 Cost: 0.424710
Epoch  300/1000 W: 0.299, b: 1.542 Cost: 0.347739
Epoch  400/1000 W: 0.363, b: 1.409 Cost: 0.289452
Epoch  500/1000 W: 0.428, b: 1.267 Cost: 0.233843
Epoch  600/1000 W: 0.493, b: 1.122 Cost: 0.183539
Epoch  700/1000 W: 0.557, b: 0.980 Cost: 0.140028
Epoch  800/1000 W: 0.619, b: 0.844 Cost: 0.103852
Epoch  900/1000 W: 0.676, b: 0.716 Cost: 0.074856
Epoch 1000/1000 W: 0.729, b: 0.599 Cost: 0.052415


In [23]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1)
    
    def forward(self,x):
        return self.linear(x)
    
#모델 초기화
model = LinearRegressionModel()

#optimizer 설정
optimizer = optim.ASGD(model.parameters(), lr=0.01)

nb_epochs=1000
for epoch in range(nb_epochs + 1):
    
    #H(x)계산
    prediction = model(x_train)
    
    #cost 계산
    cost= F.mse_loss(prediction, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        params= list(model.parameters())
        W = params[0].item()
        b= params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.114, b: 0.547 Cost: 4.589475
Epoch  100/1000 W: 0.700, b: 0.683 Cost: 0.067193
Epoch  200/1000 W: 0.764, b: 0.537 Cost: 0.041520
Epoch  300/1000 W: 0.814, b: 0.422 Cost: 0.025658
Epoch  400/1000 W: 0.854, b: 0.332 Cost: 0.015857
Epoch  500/1000 W: 0.885, b: 0.261 Cost: 0.009801
Epoch  600/1000 W: 0.910, b: 0.205 Cost: 0.006058
Epoch  700/1000 W: 0.929, b: 0.161 Cost: 0.003745
Epoch  800/1000 W: 0.944, b: 0.127 Cost: 0.002316
Epoch  900/1000 W: 0.956, b: 0.100 Cost: 0.001432
Epoch 1000/1000 W: 0.966, b: 0.078 Cost: 0.000886


#### sgd with momentum =0.9

In [24]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1)
    
    def forward(self,x):
        return self.linear(x)
    
#모델 초기화
model = LinearRegressionModel()

#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

nb_epochs=1000
for epoch in range(nb_epochs + 1):
    
   
    prediction = model(x_train) #H(x)계산
    cost= F.mse_loss(prediction, y_train)#cost 계산
    optimizer.zero_grad() #cost로 H(x) 개선
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        params= list(model.parameters())
        W = params[0].item()
        b= params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))


Epoch    0/1000 W: 0.224, b: -0.086 Cost: 3.896044
Epoch  100/1000 W: 0.999, b: 0.012 Cost: 0.000089
Epoch  200/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000
